In [58]:
%pip install pandas -U -q
%pip install numpy -U -q
%pip install yfinance -U -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [59]:
from typing import List, Dict

import pandas as pd
import numpy as np
import yfinance as yf

## Unir as ações que compõem os índices IBOV e IDIV

In [60]:
def get_tickers_from_market_index(source: str):
  market_index = pd.read_csv(source)
  market_index.drop(columns="Part. (%)", inplace=True)
  market_index.dropna(inplace=True)
  return market_index.iloc[:, 0].to_list()

def merge_tickers_from_market_indexes(*args: List[str]) -> List[str]:
  result = set()

  for market_index in args:
    result = result.union(market_index)

  return list(result)

In [61]:
ibov_composition = get_tickers_from_market_index("./assets/IBOV-composition.csv")
ibov_composition

['RRRP3',
 'ALSO3',
 'ALPA4',
 'ABEV3',
 'ARZZ3',
 'ASAI3',
 'AZUL4',
 'B3SA3',
 'BBSE3',
 'BBDC3',
 'BBDC4',
 'BRAP4',
 'BBAS3',
 'BRKM5',
 'BRFS3',
 'BPAC11',
 'CRFB3',
 'BHIA3',
 'CCRO3',
 'CMIG4',
 'CIEL3',
 'COGN3',
 'CPLE6',
 'CSAN3',
 'CPFE3',
 'CMIN3',
 'CVCB3',
 'CYRE3',
 'DXCO3',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENGI11',
 'ENEV3',
 'EGIE3',
 'EQTL3',
 'EZTC3',
 'FLRY3',
 'GGBR4',
 'GOAU4',
 'GOLL4',
 'NTCO3',
 'SOMA3',
 'HAPV3',
 'HYPE3',
 'IGTI11',
 'IRBR3',
 'ITSA4',
 'ITUB4',
 'JBSS3',
 'KLBN11',
 'RENT3',
 'LWSA3',
 'LREN3',
 'MGLU3',
 'MRFG3',
 'BEEF3',
 'MRVE3',
 'MULT3',
 'PCAR3',
 'PETR3',
 'PETR4',
 'RECV3',
 'PRIO3',
 'PETZ3',
 'RADL3',
 'RAIZ4',
 'RDOR3',
 'RAIL3',
 'SBSP3',
 'SANB11',
 'SMTO3',
 'CSNA3',
 'SLCE3',
 'SUZB3',
 'TAEE11',
 'VIVT3',
 'TIMS3',
 'TOTS3',
 'UGPA3',
 'USIM5',
 'VALE3',
 'VAMO3',
 'VBBR3',
 'WEGE3',
 'YDUQ3']

In [62]:
idiv_composition = get_tickers_from_market_index("./assets/IDIV-composition.csv")
idiv_composition

['ABCB4',
 'ALUP11',
 'AURE3',
 'B3SA3',
 'BRSR6',
 'BBSE3',
 'BRAP4',
 'BBAS3',
 'AGRO3',
 'CXSE3',
 'CMIG3',
 'CMIG4',
 'CSMG3',
 'CPLE3',
 'CPLE6',
 'CPFE3',
 'CMIN3',
 'DIRR3',
 'EGIE3',
 'FESA4',
 'FLRY3',
 'GGBR4',
 'GOAU4',
 'GRND3',
 'RANI3',
 'ITSA4',
 'JBSS3',
 'JHSF3',
 'KEPL3',
 'KLBN11',
 'LAVV3',
 'MRFG3',
 'LEVE3',
 'BEEF3',
 'PETR3',
 'PETR4',
 'PSSA3',
 'RAPT4',
 'ROMI3',
 'SANB11',
 'CSNA3',
 'TAEE11',
 'TASA4',
 'TGMA3',
 'VIVT3',
 'TRPL4',
 'TRIS3',
 'UNIP6',
 'USIM5',
 'VALE3',
 'WIZC3']

## Base de dados com com todos os atributos que compõem o fluxo de caixa

In [63]:
tickers = merge_tickers_from_market_indexes(ibov_composition, idiv_composition)

In [64]:
def format_tickers_to_yfinance(tickers: List[str]) -> List[str]:
  return [f"{ticker}.SA" for ticker in tickers]

tickers = format_tickers_to_yfinance(tickers)

In [65]:
cash_flow_data = pd.DataFrame()

for ticker in tickers:
    cashflow = yf.Ticker(ticker).quarterly_cashflow
    
    # Define o ticker como a coluna principal no MultiIndex
    cashflow.columns = pd.MultiIndex.from_product([[ticker], cashflow.columns])
    
    if cash_flow_data.empty:
        cash_flow_data = cashflow
        continue

    cash_flow_data = pd.concat([cash_flow_data, cashflow], axis=1)


In [75]:
cash_flow_data.dropna(inplace=True)
cash_flow_data

CVCB3.SA                                         \
                          2023-06-30   2023-03-31   2022-12-31   2022-09-30   
Free Cash Flow           -92966000.0 -234117000.0  374398000.0  -43475000.0   
End Cash Position        646142000.0  426211000.0  687545000.0  402354000.0   
Beginning Cash Position  426211000.0  687545000.0  402354000.0  479535000.0   
Changes In Cash          223986000.0 -258286000.0  285469000.0  -79660000.0   
Financing Cash Flow      316952000.0  -24169000.0  -88960000.0  -36185000.0   

                            RANI3.SA                                        \
                          2023-06-30   2023-03-31   2022-12-31  2022-09-30   
Free Cash Flow           -21681000.0  -76417000.0  -36195000.0 -38456000.0   
End Cash Position        368120000.0  493889000.0  735194000.0  48397000.0   
Beginning Cash Position  493889000.0  735194000.0   48397000.0  37691000.0   
Changes In Cash         -125769000.0 -241305000.0  686797000.0  10706000.0   
Financing Cash Flow     -166898000.0   17562000.0  670725000.0 -83054000.0   

                             GOAU4.SA                ...     CYRE3.SA  \
                           2023-06-30    2023-03-31  ...   2022-12-31   
Free Cash Flow           1982278000.0   981677000.0  ... -475442000.0   
End Cash Position        2556083000.0  2581882000.0  ...  129013000.0   
Beginning Cash Position  2581920000.0  2475863000.0  ...  236436000.0   
Changes In Cash            58423000.0   171115000.0  ... -107423000.0   
Financing Cash Flow     -1972052000.0  -719399000.0  ...  441502000.0   

                                           JBSS3.SA                 \
                          2022-09-30     2023-06-30     2023-03-31   
Free Cash Flow           -76282000.0   1808121000.0  -6127858000.0   
End Cash Position        236436000.0  12738712000.0   8964674000.0   
Beginning Cash Position  262973000.0   8964674000.0  13182158000.0   
Changes In Cash          -26537000.0   4092183000.0  -3912395000.0   
Financing Cash Flow      310838000.0   2175266000.0   2121373000.0   

                                                          ALSO3.SA  \
                            2022-12-31     2022-09-30   2023-06-30   
Free Cash Flow            1292162000.0   3227078000.0  353169000.0   
End Cash Position        13182158000.0  16665128000.0   34418000.0   
Beginning Cash Position  16665128000.0  19330433000.0   54829000.0   
Changes In Cash          -3142119000.0  -3088969000.0  -20411000.0   
Financing Cash Flow      -3126657000.0  -6399739000.0 -868701000.0   

                                                                
                          2023-03-31   2022-12-31   2022-09-30  
Free Cash Flow            85820000.0  324632000.0  303367000.0  
End Cash Position         54829000.0   18256000.0   18952000.0  
Beginning Cash Position   18256000.0   18952000.0   37544000.0  
Changes In Cash           36573000.0    -696000.0  -18592000.0  
Financing Cash Flow      242351000.0 -284743000.0  682825000.0  

[5 rows x 432 columns]

In [67]:
cash_flow_data.to_csv("./assets/brazilian_companies_complete_cashflow.csv")

## Base de dados  com os atributos mais relevantes para o cálculo do fluxo de caixa livre

O critério de escolha dos atributos mais relevantes é feito da seguinte maneira:

1. Cálculo da correlação de todos os atributos em relação ao atributo "Free Cash Flow" para cada ativo da base de dados;
2. Obter todos os atributos classificados como correlação forte ($cor \geq 0.7$) de cada ativo;
3. Obter os 3 atributos com maior frequência com base na etapa anterior.

Considerações importantes:

- Não foi escolhido o critério da correlação média dos atributos em relação ao "Free Cash Flow", pois pelo fato da média não ser uma medida resistente (sensível a outliers). Dessa forma, isso pode afetar o resultado de algoritmos de previsão do "Free Cash Flow", uma vez que pode ocorrer de um dos atributos selecionados como "mais relevantes" não explique a variação do "Free Cash Flow" de forma apropriada.

In [68]:
tickers = list(set([ticker for ticker, _ in cash_flow_data.columns]))
tickers

['NTCO3.SA',
 'PRIO3.SA',
 'TRIS3.SA',
 'BPAC11.SA',
 'RENT3.SA',
 'CPLE3.SA',
 'TAEE11.SA',
 'DXCO3.SA',
 'CSMG3.SA',
 'ALPA4.SA',
 'SLCE3.SA',
 'CXSE3.SA',
 'TOTS3.SA',
 'MRVE3.SA',
 'JBSS3.SA',
 'BRFS3.SA',
 'ITSA4.SA',
 'CSNA3.SA',
 'CYRE3.SA',
 'CMIG4.SA',
 'ITUB4.SA',
 'HYPE3.SA',
 'RADL3.SA',
 'PCAR3.SA',
 'BEEF3.SA',
 'ELET3.SA',
 'VAMO3.SA',
 'KEPL3.SA',
 'FESA4.SA',
 'SANB11.SA',
 'PETZ3.SA',
 'CCRO3.SA',
 'PSSA3.SA',
 'LREN3.SA',
 'ARZZ3.SA',
 'BBAS3.SA',
 'RRRP3.SA',
 'TGMA3.SA',
 'ABEV3.SA',
 'VBBR3.SA',
 'GOAU4.SA',
 'RECV3.SA',
 'LWSA3.SA',
 'CMIG3.SA',
 'FLRY3.SA',
 'CPLE6.SA',
 'UNIP6.SA',
 'RDOR3.SA',
 'AZUL4.SA',
 'CRFB3.SA',
 'UGPA3.SA',
 'WEGE3.SA',
 'BHIA3.SA',
 'WIZC3.SA',
 'CIEL3.SA',
 'CMIN3.SA',
 'EZTC3.SA',
 'CSAN3.SA',
 'BRAP4.SA',
 'ROMI3.SA',
 'BRSR6.SA',
 'ABCB4.SA',
 'BBDC4.SA',
 'EQTL3.SA',
 'RANI3.SA',
 'DIRR3.SA',
 'COGN3.SA',
 'IRBR3.SA',
 'AGRO3.SA',
 'GOLL4.SA',
 'TRPL4.SA',
 'AURE3.SA',
 'BBDC3.SA',
 'MRFG3.SA',
 'USIM5.SA',
 'YDUQ3.SA',
 'JHSF3.S

In [69]:
correlation_results = []

for ticker in tickers:
  df = cash_flow_data[ticker].transpose()
  correlation_matrix = df.corr()["Free Cash Flow"]

  # Filtra os atributos com a correlação média >= 0.7 em relação ao atributo "Free Cash Flow"
  high_corr = correlation_matrix[correlation_matrix >= 0.7]

  # Atributos com correlação forte com a coluna "Free Cash Flow"
  most_relevant = high_corr.sort_values().index.to_list()
  correlation_results.append(most_relevant)


In [70]:
import itertools
import collections


def frequency_counter(matrix: List[List[any]], n: int | None = None) -> Dict[any, int]:
    # flatten the list using chain()
    flat_list = list(itertools.chain.from_iterable(matrix))

    # count the frequencies using Counter()
    frequencies = collections.Counter(flat_list)

    if not n:
        n = len(frequencies)
    
    # Get the N most frequent attributes
    frequencies = {
        attr: frequency
        for rank, (attr, frequency) in enumerate(sorted(frequencies.items(), key=lambda item: item[1], reverse=True))
        if rank < n
    }

    return frequencies

In [71]:
frequency_counter(correlation_results)

{'Free Cash Flow': 110,
 'Changes In Cash': 48,
 'End Cash Position': 33,
 'Financing Cash Flow': 18,
 'Beginning Cash Position': 3}

In [76]:
most_relevant = frequency_counter(correlation_results, 4)
most_relevant

{'Free Cash Flow': 110,
 'Changes In Cash': 48,
 'End Cash Position': 33,
 'Financing Cash Flow': 18}

In [77]:
filter = cash_flow_data.index.isin(most_relevant)
cash_flow_data = cash_flow_data[filter]
cash_flow_data

CVCB3.SA                                         \
                      2023-06-30   2023-03-31   2022-12-31   2022-09-30   
Free Cash Flow       -92966000.0 -234117000.0  374398000.0  -43475000.0   
End Cash Position    646142000.0  426211000.0  687545000.0  402354000.0   
Changes In Cash      223986000.0 -258286000.0  285469000.0  -79660000.0   
Financing Cash Flow  316952000.0  -24169000.0  -88960000.0  -36185000.0   

                        RANI3.SA                                        \
                      2023-06-30   2023-03-31   2022-12-31  2022-09-30   
Free Cash Flow       -21681000.0  -76417000.0  -36195000.0 -38456000.0   
End Cash Position    368120000.0  493889000.0  735194000.0  48397000.0   
Changes In Cash     -125769000.0 -241305000.0  686797000.0  10706000.0   
Financing Cash Flow -166898000.0   17562000.0  670725000.0 -83054000.0   

                         GOAU4.SA                ...     CYRE3.SA  \
                       2023-06-30    2023-03-31  ...   2022-12-31   
Free Cash Flow       1982278000.0   981677000.0  ... -475442000.0   
End Cash Position    2556083000.0  2581882000.0  ...  129013000.0   
Changes In Cash        58423000.0   171115000.0  ... -107423000.0   
Financing Cash Flow -1972052000.0  -719399000.0  ...  441502000.0   

                                       JBSS3.SA                               \
                      2022-09-30     2023-06-30    2023-03-31     2022-12-31   
Free Cash Flow       -76282000.0   1808121000.0 -6127858000.0   1292162000.0   
End Cash Position    236436000.0  12738712000.0  8964674000.0  13182158000.0   
Changes In Cash      -26537000.0   4092183000.0 -3912395000.0  -3142119000.0   
Financing Cash Flow  310838000.0   2175266000.0  2121373000.0  -3126657000.0   

                                       ALSO3.SA                            \
                        2022-09-30   2023-06-30   2023-03-31   2022-12-31   
Free Cash Flow        3227078000.0  353169000.0   85820000.0  324632000.0   
End Cash Position    16665128000.0   34418000.0   54829000.0   18256000.0   
Changes In Cash      -3088969000.0  -20411000.0   36573000.0    -696000.0   
Financing Cash Flow  -6399739000.0 -868701000.0  242351000.0 -284743000.0   

                                  
                      2022-09-30  
Free Cash Flow       303367000.0  
End Cash Position     18952000.0  
Changes In Cash      -18592000.0  
Financing Cash Flow  682825000.0  

[4 rows x 432 columns]

In [78]:
cash_flow_data.to_csv("./assets/brazilian_companies_most_relevant_attr_for_free_cash_flow.csv")